INF8770 Technologies multimédias

Polytechnique Montréal

Importation des librairies

In [3]:
from PIL import Image
import os
from skimage.feature.texture import local_binary_pattern
import glob

 Matrices de Co-occurrence (MC)

In [ ]:
def desc_MC(imagePath, dx, dy, color1, color2):
  image = Image.open(imagePath)
  #width = len(image)
  #height = len(image[0])
  #result = 0

  #for column in range(0, width):
  #  for row in range(0, height):
  #    for canal in range(0, 3):
  #      if ()


  return image

print(desc_MC())

Modèles Binaires Locaux (LBP)

In [4]:
def desc_LBP(path, longueurCodes, rayon):
  return local_binary_pattern(path, longueurCodes, rayon)

Modification des données

In [10]:
query = glob.glob(os.path.join("./data", "*.jpg"))
database = glob.glob(os.path.join("./data/database", "*.jpg"))

for path_query in query:
    desc_mc = desc_MC()

['./data\\airplane_query.jpg', './data\\ball_query.jpg', './data\\car_query.jpg', './data\\cat_query.jpg', './data\\dolphin_query.jpg', './data\\face_query.jpg', './data\\lotus_query.jpg', './data\\pickles_querry.jpg', './data\\strawberry_query.jpg']
['./data/database\\airplane_1.jpg', './data/database\\airplane_2.jpg', './data/database\\airplane_3.jpg', './data/database\\airplane_4.jpg', './data/database\\airplane_5.jpg', './data/database\\ball_1.jpg', './data/database\\ball_2.jpg', './data/database\\ball_3.jpg', './data/database\\ball_4.jpg', './data/database\\ball_5.jpg', './data/database\\car_1.jpg', './data/database\\car_2.jpg', './data/database\\car_3.jpg', './data/database\\car_4.jpg', './data/database\\car_5.jpg', './data/database\\cat_1.jpg', './data/database\\cat_2.jpg', './data/database\\cat_3.jpg', './data/database\\cat_4.jpg', './data/database\\cat_5.jpg', './data/database\\dolphin_1.jpg', './data/database\\dolphin_2.jpg', './data/database\\dolphin_3.jpg', './data/database